In [1]:

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from gensim.models import Phrases,phrases
from gensim.models import Word2Vec
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from gensim.scripts import word2vec2tensor
from itertools import combinations

In [3]:
folder = '/Users/amycweng/Digital Humanities/charityTXT'
csv_data = pd.read_csv('/Users/amycweng/Digital Humanities/sermons.csv')
tcpIDs = [ _ for _ in csv_data['id']]

def getTexts(folder,search):
    texts = {}
    for file in os.listdir(folder):
        if 'NOTES' not in file: 
            name = file.split('.')[0]
            if name in search: 
                path = os.path.join(folder,file)
                with open(path,'r') as file: 
                    data = file.readlines()
                if len(data) != 0: 
                    texts[name] = data[0]
    return texts


In [4]:
all_texts = getTexts(folder,tcpIDs)
tokenized = []
for text in all_texts.values():
    words = []
    text = text.split(' ')
    for t in text:
        if len(t) != 0: words.append(t)
    tokenized.append(words)
print(len(tokenized))

70


In [5]:
# Creared word2vec model and store it
word2vec = Word2Vec(tokenized, min_count=2,sg=1)
# word2vec.wv.save_word2vec_format('/srv/data/tensor/sermons.model')
word2vec.save('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/charity.model')

In [33]:
words = ['charity','charitable','alm','bequest',
        'poor','rate','usury','usurer','give','bequeath',
        'help','money','beneficence','poverty','credit',
        'creditor','loan','lend','new','novel','novelty',
        'wealth','profit','profitable','beneficial',
        'commodity','thrift','thrifty','industry']
    
words = ['city','london','citizen']

for term in words: 
    if term in word2vec.wv: print(f'{term}: {word2vec.wv.most_similar(term,topn=5)}\n')


city: [('host', 0.8227683305740356), ('court', 0.8030896186828613), ('wilderness', 0.7994054555892944), ('palace', 0.7792544960975647), ('tabernacle', 0.778728723526001)]

london: [('yawn', 0.9597653150558472), ('sodomitical', 0.9581200480461121), ('ioseph', 0.9568988084793091), ('defiance', 0.956531286239624), ('abiram', 0.9560275673866272)]

citizen: [('faile', 0.939431369304657), ('auditor', 0.9220853447914124), ('attention', 0.9220227599143982), ('progenitor', 0.9193885326385498), ('tractable', 0.9163256287574768)]



In [9]:
words = ['relief']
sermons_model = Word2Vec.load('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/charity.model')
play_model = Word2Vec.load('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/plays.model')
for term in words: 
    if term in sermons_model.wv: print(f'sermons {term}: {sermons_model.wv.most_similar(term,topn=10)}\n')
    if term in play_model.wv: print(f'plays {term}: {play_model.wv.most_similar(term,topn=10)}\n')


sermons relief: [('recovery', 0.8479760885238647), ('absence', 0.8261793255805969), ('succour', 0.8185170292854309), ('needy', 0.8065277934074402), ('solitariness', 0.8048096299171448), ('deprive', 0.8020899891853333), ('penury', 0.801383376121521), ('relieve', 0.8009129166603088), ('weal', 0.7992798089981079), ('pinch', 0.7955381274223328)]

plays relief: [('sadness', 0.9876260757446289), ('accompany', 0.9873906970024109), ('amend', 0.9870021343231201), ('diligent', 0.9859092235565186), ('dispense', 0.985712468624115), ('allegiance', 0.9856859445571899), ('eyesight', 0.9850242137908936), ('lure', 0.9845280647277832), ('sue', 0.9837411642074585), ('provident', 0.9833546280860901)]



In [ ]:
#Create PCA table
pca = PCA(n_components=2)
pca_results = pca.fit_transform(word2vec.wv.get_normed_vectors())
pca_df = pd.DataFrame(pca_results, index=word2vec.wv.key_to_index, columns=["pc1","pc2"])


In [ ]:
# Create PCA plots
pca_df.plot(x='pc1',y='pc2',kind="scatter",figsize=(15, 10))

In [ ]:
# Create tsv file for word2vec model
word2vec2tensor.word2vec2tensor('/srv/data/tensor/relevantep.model', '/srv/data/tensor/TSV/relevantep', binary=False)

In [ ]:
# Scatter plot with label
ax = pca_df.plot(x='pc1',y='pc2',kind="scatter",figsize=(15, 10),alpha=0)
for txt in pca_df.index:
    if txt in ['charity','charities','talent','talents','profit','profits']:
        x = pca_df.pc1.loc[txt]
        y = pca_df.pc2.loc[txt]
        ax.annotate(txt, (x,y))
plt.show()

In [ ]:
#Heatmap

def cosine_similarity(pair):

    '''
    Word2Vec.wv.similarity(word1,word2)
    '''
    word1, word2 = pair
    if word1 and word2 in word2vec.wv:
        sim = word2vec.wv.similarity(word1,word2)
        return sim
    return 0


def generate_heatmap_matrix(lexicon):

    pairs = list(combinations(lexicon, 2))
    sim_scores = [cosine_similarity(pair) for pair in pairs if cosine_similarity(pair) != 0]

    sim_df = pd.DataFrame({'pair': pairs, 'similarity': sim_scores})

    sim_df['word1'] = sim_df['pair'].apply(lambda x: lexicon.index(x[0]))
    sim_df['word2'] = sim_df['pair'].apply(lambda x: lexicon.index(x[1]))

    sim_df['pair'] = list(zip(sim_df.word1, sim_df.word2, sim_df.similarity))

    df_hm = pd.DataFrame({'word1': range(len(lexicon)),
                          'word2': range(len(lexicon)),
                          'similarity': pd.Series(np.ones(len(lexicon)))})

    df_hm = df_hm.pivot(index='word1', columns='word2').fillna(0)

    for row, col, similarity in sim_df.pair:

        df_hm.iloc[col,row] = similarity

    return df_hm


def plot_heatmap(lexicon,title):

    df_hm = generate_heatmap_matrix(lexicon)

    mask = np.zeros_like(df_hm)
    mask[np.triu_indices_from(mask)] = True
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_hm,
                mask = mask,
                xticklabels = lexicon,
                yticklabels = lexicon,
                cmap="YlGnBu",
                )
    plt.xlabel('word 1')
    plt.ylabel('word 2')
    plt.title(title)
    plt.show()


In [ ]:
words = ['charity','charitable','alm','bequest',
        'poor','rate','usury','usurer','give','bequeath',
        'help','money','beneficence','poverty','credit',
        'creditor','loan','lend','new','novel','novelty',
        'wealth','profit','profitable','beneficial',
        'commodity','thrift','thrifty','industry']

plot_heatmap(words,'Similarity Between Key Word-Pairs in Charity Texts')